# WiDS Datathon 2022

## Overview

The WiDS Datathon 2022 focuses on a prediction task involving roughly 100k observations of **building energy usage records** collected over 7 years and a number of states within the United States. The dataset consists of building characteristics (e.g. floor area, facility type etc), weather data for the location of the building (e.g. annual average temperature, annual total precipitation etc) as well as the energy usage for the building and the given year, measured as **Site Energy Usage Intensity (Site EUI)**.

**Each row in the data corresponds to the a single building observed in a given year**. The task is to **predict the Site EUI for each row**, given the characteristics of the building and the weather data for the location of the building.

## Data Dictionary

  - `id`: building id
  - `Year_Factor`: anonymized year in which the weather and energy usage factors were observed
  - `State_Factor`: anonymized state in which the building is located
  - `building_class`: building classification
  - `facility_type`: building usage type
  - `floor_area`: floor area (in square feet) of the building
  - `year_built`: year in which the building was constructed
  - `energy_star_rating`: the energy star rating of the building
  - `ELEVATION`: elevation of the building location
  - `january_min_temp`: minimum temperature in January (in Fahrenheit) at the location of the building
  - `january_avg_temp`: average temperature in January (in Fahrenheit) at the location of the building
  - `january_max_temp`: maximum temperature in January (in Fahrenheit) at the location of the building
`cooling_degree_days`: cooling degree day for a given day is the number of degrees where the daily average temperature exceeds 65 degrees Fahrenheit. Each month is summed to produce an annual total at the location of the building.
`heating_degree_days`: heating degree day for a given day is the number of degrees where the daily average temperature falls under 65 degrees Fahrenheit. Each month is summed to produce an annual total at the location of the building.
  - `precipitation_inches`: annual precipitation in inches at the location of the building
  - `snowfall_inches`: annual snowfall in inches at the location of the building
  - `snowdepth_inches`: annual snow depth in inches at the location of the building
  - `avg_temp`: average temperature over a year at the location of the building
  - `days_below_30F`: total number of days below 30 degrees Fahrenheit at the location of the building
  - `days_below_20F`: total number of days below 20 degrees Fahrenheit at the location of the building
  - `days_below_10F`: total number of days below 10 degrees Fahrenheit at the location of the building
  - `days_below_0F`: total number of days below 0 degrees Fahrenheit at the location of the building
  - `days_above_80F`: total number of days above 80 degrees Fahrenheit at the location of the building
  - `days_above_90F`: total number of days above 90 degrees Fahrenheit at the location of the building
  - `days_above_100F`: total number of days above 100 degrees Fahrenheit at the location of the building
  - `days_above_110F`: total number of days above 110 degrees Fahrenheit at the location of the building
  - `direction_max_wind_speed`: wind direction for maximum wind speed at the location of the building. Given in 360-degree compass point directions (e.g. 360 = north, 180 = south, etc.).
  - `direction_peak_wind_speed`: wind direction for peak wind gust speed at the location of the building. Given in 360-degree compass point directions (e.g. 360 = north, 180 = south, etc.).
  - `max_wind_speed`: maximum wind speed at the location of the building
  - `days_with_fog`: number of days with fog at the location of the building

## Environment Setup

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max.columns', None)
pd.set_option('display.max.rows', 50)
pd.set_option('display.precision', 5)
pd.set_option('display.float_format',  '{:.5f}'.format)
warnings.simplefilter("ignore")

In [3]:
if 'notebooks' in os.getcwd():
    os.chdir('..')

DATA_PATH = os.getcwd() + '/data/'
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + '/src/')

train_fname = 'train.csv'
test_fname = 'test.csv'

In [4]:
from preprocessing import x_y_split, encoding, train_val_split

## Data Exploration

### Reading in Data

In [5]:
train_df = pd.read_csv(DATA_PATH + train_fname, header='infer', low_memory=False)
train_df.columns = [x.lower() for x in train_df.columns]
train_df.set_index('id', inplace=True)
print(train_df.shape)
train_df.head()

(75757, 63)


,year_factor,state_factor,building_class,facility_type,floor_area,year_built,energy_star_rating,elevation,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30f,days_below_20f,days_below_10f,days_below_0f,days_above_80f,days_above_90f,days_above_100f,days_above_110f,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.00000,1942.00000,11.00000,2.40000,36,50.50000,68,35,50.58929,73,40,53.69355,80,41,55.50000,78,46,56.85484,84,50,60.50000,90,52,62.72581,84,52,62.16129,85,52,64.65000,90,47,63.01613,83,43,53.80000,72,36,49.27419,71,115,2960,16.59000,0.00000,0,56.97260,0,0,0,0,14,0,0,0,1.00000,1.00000,1.00000,nan,248.68262
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.00000,1955.00000,45.00000,1.80000,36,50.50000,68,35,50.58929,73,40,53.69355,80,41,55.50000,78,46,56.85484,84,50,60.50000,90,52,62.72581,84,52,62.16129,85,52,64.65000,90,47,63.01613,83,43,53.80000,72,36,49.27419,71,115,2960,16.59000,0.00000,0,56.97260,0,0,0,0,14,0,0,0,1.00000,nan,1.00000,12.00000,26.50015
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.00000,1951.00000,97.00000,1.80000,36,50.50000,68,35,50.58929,73,40,53.69355,80,41,55.50000,78,46,56.85484,84,50,60.50000,90,52,62.72581,84,52,62.16129,85,52,64.65000,90,47,63.01613,83,43,53.80000,72,36,49.27419,71,115,2960,16.59000,0.00000,0,56.97260,0,0,0,0,14,0,0,0,1.00000,nan,1.00000,12.00000,24.69362
3,1,State_1,Commercial,Education_Other_classroom,55325.00000,1980.00000,46.00000,1.80000,36,50.50000,68,35,50.58929,73,40,53.69355,80,41,55.50000,78,46,56.85484,84,50,60.50000,90,52,62.72581,84,52,62.16129,85,52,64.65000,90,47,63.01613,83,43,53.80000,72,36,49.27419,71,115,2960,16.59000,0.00000,0,56.97260,0,0,0,0,14,0,0,0,1.00000,nan,1.00000,12.00000,48.40693
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.00000,1985.00000,100.00000,2.40000,36,50.50000,68,35,50.58929,73,40,53.69355,80,41,55.50000,78,46,56.85484,84,50,60.50000,90,52,62.72581,84,52,62.16129,85,52,64.65000,90,47,63.01613,83,43,53.80000,72,36,49.27419,71,115,2960,16.59000,0.00000,0,56.97260,0,0,0,0,14,0,0,0,1.00000,1.00000,1.00000,nan,3.89940


In [6]:
test_df = pd.read_csv(DATA_PATH + test_fname, header='infer', low_memory=False)
test_df.columns = [x.lower() for x in test_df.columns]
test_df.set_index('id', inplace=True)
print(test_df.shape)
test_df.head()

(9705, 62)


,year_factor,state_factor,building_class,facility_type,floor_area,year_built,energy_star_rating,elevation,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30f,days_below_20f,days_below_10f,days_below_0f,days_above_80f,days_above_90f,days_above_100f,days_above_110f,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
75757,7,State_1,Commercial,Grocery_store_or_food_market,28484.00000,1994.00000,37.00000,2.40000,38,50.59677,64,40,54.48214,66,42,56.93548,77,45,58.45000,78,49,60.90323,87,51,63.15000,97,53,64.25806,85,53,65.85484,88,54,69.76667,104,48,64.19355,91,43,58.05000,69,38,51.88710,66,342,2112,25.24000,0.00000,0,59.89452,0,0,0,0,29,5,2,0,nan,nan,nan,nan
75758,7,State_1,Commercial,Grocery_store_or_food_market,21906.00000,1961.00000,55.00000,45.70000,38,50.59677,64,40,54.48214,66,42,56.93548,77,45,58.45000,78,49,60.90323,87,51,63.15000,97,53,64.25806,85,53,65.85484,88,54,69.76667,104,48,64.19355,91,43,58.05000,69,38,51.88710,66,342,2112,25.24000,0.00000,0,59.89452,0,0,0,0,29,5,2,0,nan,nan,nan,nan
75759,7,State_1,Commercial,Grocery_store_or_food_market,16138.00000,1950.00000,1.00000,59.10000,38,50.59677,64,40,54.48214,66,42,56.93548,77,45,58.45000,78,49,60.90323,87,51,63.15000,97,53,64.25806,85,53,65.85484,88,54,69.76667,104,48,64.19355,91,43,58.05000,69,38,51.88710,66,342,2112,25.24000,0.00000,0,59.89452,0,0,0,0,29,5,2,0,nan,nan,nan,nan
75760,7,State_1,Commercial,Grocery_store_or_food_market,97422.00000,1971.00000,34.00000,35.40000,38,50.59677,64,40,54.48214,66,42,56.93548,77,45,58.45000,78,49,60.90323,87,51,63.15000,97,53,64.25806,85,53,65.85484,88,54,69.76667,104,48,64.19355,91,43,58.05000,69,38,51.88710,66,342,2112,25.24000,0.00000,0,59.89452,0,0,0,0,29,5,2,0,nan,nan,nan,nan
75761,7,State_1,Commercial,Grocery_store_or_food_market,61242.00000,1942.00000,35.00000,1.80000,38,50.59677,64,40,54.48214,66,42,56.93548,77,45,58.45000,78,49,60.90323,87,51,63.15000,97,53,64.25806,85,53,65.85484,88,54,69.76667,104,48,64.19355,91,43,58.05000,69,38,51.88710,66,342,2112,25.24000,0.00000,0,59.89452,0,0,0,0,29,5,2,0,340.00000,330.00000,22.80000,126.00000


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75757 entries, 0 to 75756
Data columns (total 63 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year_factor                75757 non-null  int64  
 1   state_factor               75757 non-null  object 
 2   building_class             75757 non-null  object 
 3   facility_type              75757 non-null  object 
 4   floor_area                 75757 non-null  float64
 5   year_built                 73920 non-null  float64
 6   energy_star_rating         49048 non-null  float64
 7   elevation                  75757 non-null  float64
 8   january_min_temp           75757 non-null  int64  
 9   january_avg_temp           75757 non-null  float64
 10  january_max_temp           75757 non-null  int64  
 11  february_min_temp          75757 non-null  int64  
 12  february_avg_temp          75757 non-null  float64
 13  february_max_temp          75757 non-null  int

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9705 entries, 75757 to 85461
Data columns (total 62 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year_factor                9705 non-null   int64  
 1   state_factor               9705 non-null   object 
 2   building_class             9705 non-null   object 
 3   facility_type              9705 non-null   object 
 4   floor_area                 9705 non-null   float64
 5   year_built                 9613 non-null   float64
 6   energy_star_rating         7451 non-null   float64
 7   elevation                  9705 non-null   float64
 8   january_min_temp           9705 non-null   int64  
 9   january_avg_temp           9705 non-null   float64
 10  january_max_temp           9705 non-null   int64  
 11  february_min_temp          9705 non-null   int64  
 12  february_avg_temp          9705 non-null   float64
 13  february_max_temp          9705 non-null   

Columns with nulls - 

  - `year_built`
  - `energy_star_rating`
  - `direction_max_wind_speed`
  - `direction_peak_wind_speed`
  - `max_wind_speed`
  - `days_with_fog`

### Categorical Attributes

  - `year_factor`
  - `state_factor`
  - `building_class`
  - `facility_type`

In [9]:
cat_cols = ['year_factor', 'state_factor', 'building_class', 'facility_type']
for col in cat_cols:
    print(col, train_df[col].nunique())
    print(train_df[col].value_counts())
    print(2*len(col)*'-')

year_factor 6
6    22449
5    18308
4    12946
3    10879
2     9058
1     2117
Name: year_factor, dtype: int64
----------------------
state_factor 7
State_6     50840
State_11     6412
State_1      5618
State_2      4871
State_4      4300
State_8      3701
State_10       15
Name: state_factor, dtype: int64
------------------------
building_class 2
Residential    43558
Commercial     32199
Name: building_class, dtype: int64
----------------------------
facility_type 60
Multifamily_Uncategorized              39455
Office_Uncategorized                   12512
Education_Other_classroom               3860
Lodging_Hotel                           2098
2to4_Unit_Building                      1893
                                       ...  
Food_Service_Other                        17
Mixed_Use_Predominantly_Residential        9
Service_Drycleaning_or_Laundry             9
Public_Assembly_Stadium                    9
Lodging_Uncategorized                      5
Name: facility_type, Length: 60

### Numerical Attributes

In [10]:
num_cols = [x for x in train_df.columns if x not in cat_cols and x!='id']
for col in num_cols:
    print(col)
    print(train_df[col].describe().loc[['mean', '50%']])

floor_area
mean   165983.86586
50%     91367.00000
Name: floor_area, dtype: float64
year_built
mean   1952.30676
50%    1951.00000
Name: year_built, dtype: float64
energy_star_rating
mean   61.04861
50%    67.00000
Name: energy_star_rating, dtype: float64
elevation
mean   39.50632
50%    25.00000
Name: elevation, dtype: float64
january_min_temp
mean   11.43234
50%    11.00000
Name: january_min_temp, dtype: float64
january_avg_temp
mean   34.31047
50%    34.45161
Name: january_avg_temp, dtype: float64
january_max_temp
mean   59.05495
50%    59.00000
Name: january_max_temp, dtype: float64
february_min_temp
mean   11.72057
50%     9.00000
Name: february_min_temp, dtype: float64
february_avg_temp
mean   35.52684
50%    34.10714
Name: february_avg_temp, dtype: float64
february_max_temp
mean   58.48628
50%    61.00000
Name: february_max_temp, dtype: float64
march_min_temp
mean   21.60628
50%    25.00000
Name: march_min_temp, dtype: float64
march_avg_temp
mean   44.46929
50%    44.51613
Name:

### XGBoost

In [11]:
train_df.drop(["year_factor"], axis=1, inplace=True)
test_df.drop(["year_factor"], axis=1, inplace=True)

In [12]:
X, y = x_y_split(train_df, col_target="site_eui", aligned=False)
X_train, X_val, y_train, y_val = train_val_split(X, y)

In [13]:
X_train.head()

,state_factor,building_class,facility_type,floor_area,year_built,energy_star_rating,elevation,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30f,days_below_20f,days_below_10f,days_below_0f,days_above_80f,days_above_90f,days_above_100f,days_above_110f,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66822,State_8,Commercial,Education_Other_classroom,70230.00000,1927.00000,21.00000,12.20000,3,29.25806,62,14,33.48077,63,10,38.01613,69,33,55.33333,84,47,65.96774,90,53,75.40000,96,62,78.53226,95,59,75.38710,92,51,71.20000,94,40,61.09677,80,22,44.21667,71,26,41.27419,64,1437,4614,48.82000,42.10000,119,56.00689,65,26,4,0,93,20,0,0,nan,nan,nan,nan
19479,State_6,Residential,Multifamily_Uncategorized,1108700.00000,nan,nan,42.70000,13,37.32258,62,20,40.87931,62,25,50.93548,78,38,54.76667,88,49,65.09677,89,51,70.98333,94,61,78.75806,100,61,76.69355,91,53,68.80000,91,38,58.04839,78,31,43.86667,66,28,41.46774,62,1277,3978,38.51000,9.60000,10,57.36612,22,5,0,0,72,17,0,0,1.00000,1.00000,1.00000,99.00000
11197,State_4,Residential,Mixed_Use_Commercial_and_Residential,55466.00000,1932.00000,100.00000,165.50000,-8,21.93548,46,-13,15.69643,44,0,34.16129,72,27,48.20000,78,35,60.24194,86,41,65.36667,91,51,70.67742,94,50,71.56452,94,46,68.86667,93,29,54.06452,78,12,44.73333,73,15,39.69355,62,769,6226,41.75000,44.80000,426,49.80000,96,53,31,15,64,8,0,0,nan,nan,nan,nan
63760,State_6,Commercial,Education_Other_classroom,90100.00000,1995.00000,53.00000,25.00000,11,34.45161,59,-1,37.70690,61,26,48.87097,79,26,53.33333,82,43,62.79032,92,52,72.25000,88,63,78.67742,96,61,79.17742,96,54,71.80000,91,38,58.75806,85,34,49.76667,72,17,38.33871,60,1489,4252,42.17000,36.00000,101,57.21585,40,11,2,1,96,17,0,0,1.00000,1.00000,1.00000,nan
15423,State_6,Commercial,Office_Uncategorized,97554.00000,1920.00000,76.00000,42.70000,6,29.67742,53,15,36.03571,67,20,42.32258,77,35,54.30000,83,46,64.48387,89,56,72.31667,95,63,80.24194,104,59,75.27419,94,51,70.03333,86,33,57.09677,84,36,51.88333,70,22,43.30645,62,1331,4335,72.81000,44.70000,418,56.51918,54,11,1,0,84,16,1,0,1.00000,1.00000,1.00000,12.00000


In [14]:
X_val.head()

,state_factor,building_class,facility_type,floor_area,year_built,energy_star_rating,elevation,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30f,days_below_20f,days_below_10f,days_below_0f,days_above_80f,days_above_90f,days_above_100f,days_above_110f,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29922,State_6,Commercial,Lodging_Hotel,254829.00000,1990.00000,nan,15.20000,11,35.08065,61,17,33.91071,55,27,40.14516,59,33,52.98333,82,42,62.79032,90,53,72.65000,92,64,79.83871,98,59,74.59677,90,47,67.86667,96,40,60.19355,86,23,45.25000,70,19,38.54839,71,1272,4670,46.32000,29.60000,64,55.44932,50,11,0,0,75,11,0,0,nan,nan,nan,nan
55120,State_6,Commercial,Education_Other_classroom,68480.00000,1928.00000,97.00000,3.40000,11,34.45161,59,-1,37.70690,61,26,48.87097,79,26,53.33333,82,43,62.79032,92,52,72.25000,88,63,78.67742,96,61,79.17742,96,54,71.80000,91,38,58.75806,85,34,49.76667,72,17,38.33871,60,1489,4252,42.17000,36.00000,101,57.21585,40,11,2,1,96,17,0,0,1.00000,1.00000,1.00000,nan
52830,State_6,Residential,Multifamily_Uncategorized,103724.00000,1941.00000,65.00000,10.70000,11,34.45161,59,-1,37.70690,61,26,48.87097,79,26,53.33333,82,43,62.79032,92,52,72.25000,88,63,78.67742,96,61,79.17742,96,54,71.80000,91,38,58.75806,85,34,49.76667,72,17,38.33871,60,1489,4252,42.17000,36.00000,101,57.21585,40,11,2,1,96,17,0,0,nan,nan,nan,nan
47809,State_6,Commercial,Office_Uncategorized,231938.00000,1957.00000,78.00000,21.30000,8,29.85484,56,2,23.94643,43,12,38.09677,62,32,54.25000,80,48,68.51613,88,50,71.18333,90,64,78.79032,96,63,78.96774,95,56,74.45000,97,35,58.04839,78,32,52.78333,74,34,50.77419,72,1581,4460,40.97000,49.10000,463,56.83562,63,34,9,0,106,12,0,0,nan,nan,nan,nan
4663,State_1,Commercial,Office_Bank_or_other_financial,13040.00000,1931.00000,86.00000,2.40000,37,52.69355,62,42,56.94828,75,47,57.72581,76,47,60.28333,86,51,61.06452,87,51,63.35000,87,53,62.91935,81,53,63.79032,76,52,64.80000,94,51,63.35484,84,44,58.28333,75,36,50.66129,63,142,2005,21.39000,0.00000,0,59.64891,0,0,0,0,16,2,0,0,nan,nan,nan,nan


In [15]:
y_train.head()

,site_eui
id,
66822,89.62537
19479,124.55874
11197,26.62242
63760,64.62295
15423,94.49891


In [16]:
y_val.head()

,site_eui
id,
29922,164.20672
55120,46.37295
52830,84.06967
47809,69.31123
4663,41.08743


In [17]:
cat_cols = ['state_factor', 'building_class', 'facility_type']

In [18]:
X_train = encoding(X_train, cat_cols)
X_train.head()

,floor_area,year_built,energy_star_rating,elevation,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30f,days_below_20f,days_below_10f,days_below_0f,days_above_80f,days_above_90f,days_above_100f,days_above_110f,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,state_factor_State_1,state_factor_State_10,state_factor_State_11,state_factor_State_2,state_factor_State_4,state_factor_State_6,state_factor_State_8,building_class_Commercial,building_class_Residential,facility_type_2to4_Unit_Building,facility_type_5plus_Unit_Building,facility_type_Commercial_Other,facility_type_Commercial_Unknown,facility_type_Data_Center,facility_type_Education_College_or_university,facility_type_Education_Other_classroom,facility_type_Education_Preschool_or_daycare,facility_type_Education_Uncategorized,facility_type_Food_Sales,facility_type_Food_Service_Other,facility_type_Food_Service_Restaurant_or_cafeteria,facility_type_Food_Service_Uncategorized,facility_type_Grocery_store_or_food_market,facility_type_Health_Care_Inpatient,facility_type_Health_Care_Outpatient_Clinic,facility_type_Health_Care_Outpatient_Uncategorized,facility_type_Health_Care_Uncategorized,facility_type_Industrial,facility_type_Laboratory,facility_type_Lodging_Dormitory_or_fraternity_sorority,facility_type_Lodging_Hotel,facility_type_Lodging_Other,facility_type_Lodging_Uncategorized,facility_type_Mixed_Use_Commercial_and_Residential,facility_type_Mixed_Use_Predominantly_Commercial,facility_type_Mixed_Use_Predominantly_Residential,facility_type_Multifamily_Uncategorized,facility_type_Nursing_Home,facility_type_Office_Bank_or_other_financial,facility_type_Office_Medical_non_diagnostic,facility_type_Office_Mixed_use,facility_type_Office_Uncategorized,facility_type_Parking_Garage,facility_type_Public_Assembly_Drama_theater,facility_type_Public_Assembly_Entertainment_culture,facility_type_Public_Assembly_Library,facility_type_Public_Assembly_Movie_Theater,facility_type_Public_Assembly_Other,facility_type_Public_Assembly_Recreation,facility_type_Public_Assembly_Social_meeting,facility_type_Public_Assembly_Stadium,facility_type_Public_Assembly_Uncategorized,facility_type_Public_Safety_Courthouse,facility_type_Public_Safety_Fire_or_police_station,facility_type_Public_Safety_Penitentiary,facility_type_Public_Safety_Uncategorized,facility_type_Religious_worship,facility_type_Retail_Enclosed_mall,facility_type_Retail_Strip_shopping_mall,facility_type_Retail_Uncategorized,facility_type_Retail_Vehicle_dealership_showroom,facility_type_Service_Drycleaning_or_Laundry,facility_type_Service_Uncategorized,facility_type_Service_Vehicle_service_repair_shop,facility_type_Warehouse_Distribution_or_Shipping_center,facility_type_Warehouse_Nonrefrigerated,facility_type_Warehouse_Refrigerated,facility_type_Warehouse_Selfstorage,facility_type_Warehouse_Uncategorized
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66822,70230.00000,1927.00000,21.00000,12.20000,3,29.25806,62,14,33.48077,63,10,38.01613,69,33,55.33333,84,47,65.96774,90,53,75.40000,96,62,78.53226,95,59,75.38710,92,51,71.20000,94,40,61.09677,80,22,44.21667,71,26,41.27419,64,1437,4614,48.82000,42.10000,119,56.00689,65,26,4,0,93,20,0,0,nan,nan,nan,nan,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
X_val = encoding(X_val, cat_cols)
X_val.head()

,floor_area,year_built,energy_star_rating,elevation,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30f,days_below_20f,days_below_10f,days_below_0f,days_above_80f,days_above_90f,days_above_100f,days_above_110f,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,state_factor_State_1,state_factor_State_10,state_factor_State_11,state_factor_State_2,state_factor_State_4,state_factor_State_6,state_factor_State_8,building_class_Commercial,building_class_Residential,facility_type_2to4_Unit_Building,facility_type_5plus_Unit_Building,facility_type_Commercial_Other,facility_type_Commercial_Unknown,facility_type_Data_Center,facility_type_Education_College_or_university,facility_type_Education_Other_classroom,facility_type_Education_Preschool_or_daycare,facility_type_Education_Uncategorized,facility_type_Food_Sales,facility_type_Food_Service_Other,facility_type_Food_Service_Restaurant_or_cafeteria,facility_type_Food_Service_Uncategorized,facility_type_Grocery_store_or_food_market,facility_type_Health_Care_Inpatient,facility_type_Health_Care_Outpatient_Clinic,facility_type_Health_Care_Outpatient_Uncategorized,facility_type_Health_Care_Uncategorized,facility_type_Industrial,facility_type_Laboratory,facility_type_Lodging_Dormitory_or_fraternity_sorority,facility_type_Lodging_Hotel,facility_type_Lodging_Other,facility_type_Lodging_Uncategorized,facility_type_Mixed_Use_Commercial_and_Residential,facility_type_Mixed_Use_Predominantly_Commercial,facility_type_Mixed_Use_Predominantly_Residential,facility_type_Multifamily_Uncategorized,facility_type_Nursing_Home,facility_type_Office_Bank_or_other_financial,facility_type_Office_Medical_non_diagnostic,facility_type_Office_Mixed_use,facility_type_Office_Uncategorized,facility_type_Parking_Garage,facility_type_Public_Assembly_Drama_theater,facility_type_Public_Assembly_Entertainment_culture,facility_type_Public_Assembly_Library,facility_type_Public_Assembly_Movie_Theater,facility_type_Public_Assembly_Other,facility_type_Public_Assembly_Recreation,facility_type_Public_Assembly_Social_meeting,facility_type_Public_Assembly_Stadium,facility_type_Public_Assembly_Uncategorized,facility_type_Public_Safety_Courthouse,facility_type_Public_Safety_Fire_or_police_station,facility_type_Public_Safety_Penitentiary,facility_type_Public_Safety_Uncategorized,facility_type_Religious_worship,facility_type_Retail_Enclosed_mall,facility_type_Retail_Strip_shopping_mall,facility_type_Retail_Uncategorized,facility_type_Retail_Vehicle_dealership_showroom,facility_type_Service_Drycleaning_or_Laundry,facility_type_Service_Uncategorized,facility_type_Service_Vehicle_service_repair_shop,facility_type_Warehouse_Distribution_or_Shipping_center,facility_type_Warehouse_Nonrefrigerated,facility_type_Warehouse_Refrigerated,facility_type_Warehouse_Selfstorage,facility_type_Warehouse_Uncategorized
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29922,254829.00000,1990.00000,nan,15.20000,11,35.08065,61,17,33.91071,55,27,40.14516,59,33,52.98333,82,42,62.79032,90,53,72.65000,92,64,79.83871,98,59,74.59677,90,47,67.86667,96,40,60.19355,86,23,45.25000,70,19,38.54839,71,1272,4670,46.32000,29.60000,64,55.44932,50,11,0,0,75,11,0,0,nan,nan,nan,nan,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
X_train.drop(["state_factor_State_6"], axis=1, inplace=True)
X_val.drop(["state_factor_State_6"], axis=1, inplace=True)

In [21]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [22]:
eval_set = [(X_train, y_train), (X_val, y_val)]
model = XGBRegressor()
model.fit(X_train, y_train, eval_metric="rmse", eval_set=eval_set, verbose=True)
print(model)

[0]	validation_0-rmse:78.40442	validation_1-rmse:77.65055
[1]	validation_0-rmse:64.33566	validation_1-rmse:63.77132
[2]	validation_0-rmse:55.79013	validation_1-rmse:55.31369
[3]	validation_0-rmse:50.86290	validation_1-rmse:50.55011
[4]	validation_0-rmse:47.68791	validation_1-rmse:47.64352
[5]	validation_0-rmse:45.96593	validation_1-rmse:46.07508
[6]	validation_0-rmse:44.70985	validation_1-rmse:44.94727
[7]	validation_0-rmse:44.02307	validation_1-rmse:44.36613
[8]	validation_0-rmse:43.20827	validation_1-rmse:43.79585
[9]	validation_0-rmse:42.83675	validation_1-rmse:43.58898
[10]	validation_0-rmse:42.40369	validation_1-rmse:43.17939
[11]	validation_0-rmse:42.13249	validation_1-rmse:42.99712
[12]	validation_0-rmse:41.54060	validation_1-rmse:42.80873
[13]	validation_0-rmse:41.37103	validation_1-rmse:42.65352
[14]	validation_0-rmse:41.23980	validation_1-rmse:42.58826
[15]	validation_0-rmse:40.95542	validation_1-rmse:42.47368
[16]	validation_0-rmse:40.70458	validation_1-rmse:42.40264
[17]	va

In [23]:
# make predictions for test data
y_pred = model.predict(X_val)
y_pred

array([119.7991  ,  51.209984,  77.94386 , ...,  88.71582 , 138.47911 ,
        95.87642 ], dtype=float32)

In [24]:
np.sqrt(mean_squared_error(y_val, y_pred))

40.02778257735514

### Submission

In [25]:
test_df = encoding(test_df, cat_cols)
test_df.head()

,floor_area,year_built,energy_star_rating,elevation,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30f,days_below_20f,days_below_10f,days_below_0f,days_above_80f,days_above_90f,days_above_100f,days_above_110f,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,state_factor_State_1,state_factor_State_10,state_factor_State_11,state_factor_State_2,state_factor_State_4,state_factor_State_8,building_class_Commercial,building_class_Residential,facility_type_2to4_Unit_Building,facility_type_5plus_Unit_Building,facility_type_Commercial_Other,facility_type_Commercial_Unknown,facility_type_Data_Center,facility_type_Education_College_or_university,facility_type_Education_Other_classroom,facility_type_Education_Preschool_or_daycare,facility_type_Education_Uncategorized,facility_type_Food_Sales,facility_type_Food_Service_Other,facility_type_Food_Service_Restaurant_or_cafeteria,facility_type_Food_Service_Uncategorized,facility_type_Grocery_store_or_food_market,facility_type_Health_Care_Inpatient,facility_type_Health_Care_Outpatient_Clinic,facility_type_Health_Care_Outpatient_Uncategorized,facility_type_Health_Care_Uncategorized,facility_type_Industrial,facility_type_Laboratory,facility_type_Lodging_Dormitory_or_fraternity_sorority,facility_type_Lodging_Hotel,facility_type_Lodging_Other,facility_type_Lodging_Uncategorized,facility_type_Mixed_Use_Commercial_and_Residential,facility_type_Mixed_Use_Predominantly_Commercial,facility_type_Mixed_Use_Predominantly_Residential,facility_type_Multifamily_Uncategorized,facility_type_Nursing_Home,facility_type_Office_Bank_or_other_financial,facility_type_Office_Medical_non_diagnostic,facility_type_Office_Mixed_use,facility_type_Office_Uncategorized,facility_type_Parking_Garage,facility_type_Public_Assembly_Drama_theater,facility_type_Public_Assembly_Entertainment_culture,facility_type_Public_Assembly_Library,facility_type_Public_Assembly_Movie_Theater,facility_type_Public_Assembly_Other,facility_type_Public_Assembly_Recreation,facility_type_Public_Assembly_Social_meeting,facility_type_Public_Assembly_Stadium,facility_type_Public_Assembly_Uncategorized,facility_type_Public_Safety_Courthouse,facility_type_Public_Safety_Fire_or_police_station,facility_type_Public_Safety_Penitentiary,facility_type_Public_Safety_Uncategorized,facility_type_Religious_worship,facility_type_Retail_Enclosed_mall,facility_type_Retail_Strip_shopping_mall,facility_type_Retail_Uncategorized,facility_type_Retail_Vehicle_dealership_showroom,facility_type_Service_Drycleaning_or_Laundry,facility_type_Service_Uncategorized,facility_type_Service_Vehicle_service_repair_shop,facility_type_Warehouse_Distribution_or_Shipping_center,facility_type_Warehouse_Nonrefrigerated,facility_type_Warehouse_Refrigerated,facility_type_Warehouse_Selfstorage,facility_type_Warehouse_Uncategorized
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
75757,28484.00000,1994.00000,37.00000,2.40000,38,50.59677,64,40,54.48214,66,42,56.93548,77,45,58.45000,78,49,60.90323,87,51,63.15000,97,53,64.25806,85,53,65.85484,88,54,69.76667,104,48,64.19355,91,43,58.05000,69,38,51.88710,66,342,2112,25.24000,0.00000,0,59.89452,0,0,0,0,29,5,2,0,nan,nan,nan,nan,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
y_pred_test = model.predict(test_df)
y_pred_test

array([239.23476 , 258.10782 , 213.40337 , ...,  76.55459 ,  85.85267 ,
        31.628738], dtype=float32)

In [27]:
submissions_df = pd.DataFrame({'id': test_df.index, 'site_eui': y_pred_test})
submissions_df.to_csv(DATA_PATH +'submissions.csv', index=False, header=True)

In [28]:
from submission import publish_submission

In [29]:
publish_submission("widsdatathon2022", "API Submission Test", DATA_PATH +'submissions.csv')

100%|██████████| 156k/156k [00:01<00:00, 98.8kB/s]
